Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/4956778/8344638/a2a6aa289fce8461958dc287f1dab799/dataset-cover.jpg?t=2024-05-07-09-36-53" alt="IMG">
</p>

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
This dataset explores how weather conditions impact renewable energy generation. <br><br>Spanning from January 1, 2017, to August 31, 2022, the dataset provides climate data such as temperature, pressure, wind speed, and sunlight duration at 15-minute intervals. <br><br>By incorporating variables like GHI and SunlightTime, it enables the prediction of solar energy production.


- DataSource: https://www.kaggle.com/datasets/pythonafroz/renewable-power-generation-and-weather-conditions/data
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 

<details>
    <summary>More Columns Info</summary>
    17 columns in the dataset, below is <u>description of main features</u>:

    *  (1) Time: The timestamp of the recorded data in the format of YYYY-MM-DD HH:MM:SS.
        
    *  (2) Energy delta[Wh]: The difference in energy consumption in Watt-hours (Wh) from the previous timestamp to the current timestamp.
        
    *  (3) GHI: Global Horizontal Irradiance in Watts per square meter (W/m²) measured by a pyranometer.
        
    *  (4) temp: The temperature in degrees Celsius (°C) measured at the same height as the pyranometer.

    *  (5) pressure: The atmospheric pressure in hectopascals (hPa) measured at the same height as the pyranometer.

    *  (6) humidity: The relative humidity in percentage (%) measured at the same height as the pyranometer.

    *  (7) wind_speed: The wind speed in meters per second (m/s) measured at the same height as the pyranometer.

    *  (8) rain_1h: The amount of precipitation in millimeters (mm) measured over the past hour.
    
    *  (9) snow_1h: The amount of snowfall in millimeters.

    *  (10) clouds_all: The cloud situation.
</details>    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using GPT, Pytorch. <br><br>
    The Target Column used here is <b>Energy delta[Wh]</b>
</h4>

 

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
df = pd.read_csv(path_append + '../data/Renewable Power Generation and weather Conditions/Renewable.csv')
df.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
0,2017-01-01 00:00:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
1,2017-01-01 00:15:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
2,2017-01-01 00:30:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
3,2017-01-01 00:45:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
4,2017-01-01 01:00:00,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:

df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')

# Extract time in seconds from the datetime object
df['SecondsOfDay'] = df['Time'].dt.hour * 3600 + df['Time'].dt.minute * 60 + df['Time'].dt.second

# Optionally drop the original Time column
df.drop(['Time'], axis=1, inplace=True)

# Display the first few rows to verify the changes
df.head()

,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month,SecondsOfDay
0,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,0
1,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,900
2,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,1800
3,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,2700
4,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1,3600


In [4]:
from tools.data_frame import process_dataframe
target_column = ['Energy delta[Wh]']
one_hot_columns = ['weather_type', 'month']
minmax_columns = df.columns.difference(target_column)
robust_columns = target_column
df = process_dataframe(df, target_column, one_hot_columns = one_hot_columns, minmax_columns = minmax_columns, robust_columns = robust_columns) 
num_features = df.shape[1]

print(f"Number of features after scaling: {num_features}")


Number of features after scaling: 32


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196776 entries, 0 to 196775
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   GHI                     196776 non-null  float64
 1   temp                    196776 non-null  float64
 2   pressure                196776 non-null  float64
 3   humidity                196776 non-null  float64
 4   wind_speed              196776 non-null  float64
 5   rain_1h                 196776 non-null  float64
 6   snow_1h                 196776 non-null  float64
 7   clouds_all              196776 non-null  float64
 8   isSun                   196776 non-null  float64
 9   sunlightTime            196776 non-null  float64
 10  dayLength               196776 non-null  float64
 11  SunlightTime/daylength  196776 non-null  float64
 12  hour                    196776 non-null  float64
 13  SecondsOfDay            196776 non-null  float64
 14  weather_type_1      

In [6]:
df.head()

,GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,Energy delta[Wh]
0,0.0,0.347328,0.628571,1.0,0.342657,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.347328,0.628571,1.0,0.342657,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.347328,0.628571,1.0,0.342657,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.347328,0.628571,1.0,0.342657,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.349237,0.614286,1.0,0.363636,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
import torch
from torch.utils.data import Dataset
import random

class SequentialDataset(Dataset):
    def __init__(self, df, indices, max_window_size, num_classes):
        self.df = df
        self.indices = indices
        self.max_window_size = max_window_size
        self.min_window_size = max_window_size // 2
        self.num_classes = num_classes

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        start_idx = self.indices[idx]
        window_size = random.randint(self.min_window_size, self.max_window_size)
        # window_size = self.max_window_size
        end_idx = min(start_idx + window_size, len(self.df))

        seq = self.df.iloc[start_idx:end_idx]
        X, y = seq.values[:-1, :], seq.values[1:, -1:] # target column is the last column
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y

In [8]:
from random import shuffle
from sklearn.model_selection import train_test_split

# Assume 'df' is your DataFrame and 'event' is the column containing labels

def generate_indices(df, max_window_size):
    indices = []
    max_index = len(df) - max_window_size + 1  # Calculate the maximum starting index
    
    for i in range(max_index):
        # Check if all labels in the window are the same
        indices.append(i)
    
    return indices
# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
# Assuming 'df' and 'num_classes' are defined
max_window_size = 48
train_indices = generate_indices(train_df, max_window_size)
test_indices = generate_indices(test_df, max_window_size)

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
print('Train indices: ', len(train_indices))
print('Test indices: ', len(test_indices))

trainset = SequentialDataset(df=train_df, indices=train_indices, max_window_size=max_window_size, num_classes=num_features)
testset = SequentialDataset(df=test_df, indices=test_indices, max_window_size=max_window_size, num_classes=num_features)

train df shape:  (137743, 32)
test df shape:  (59033, 32)
Train indices:  137696
Test indices:  58986


<a id="3"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>3.</b> Modeling</b></h1>

In [9]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

data_config = DataConfig(dataset_name = 'renewable-power-gen-prediction', task_type='regression', obs_shape=[num_features], label_size=1)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'gpt', encoder_network = 'none')

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False) 

In [10]:
trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2151 [00:00<?, ?it/s]

[0/100][50/2151][Time 10.78]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Gpt
Inf: 0.1245	Gen: 0.2498	Rec: 0.2344	E: 0.0680	R: 0.0511	P: 0.1652
--------------------Test Metrics------------------------
mse: 43.0515
mae: 6.2492
r2: -10.1973

[0/100][100/2151][Time 10.01]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Gpt
Inf: 0.0760	Gen: 0.1597	Rec: 0.1241	E: 0.0287	R: 0.0033	P: 0.0469
--------------------Test Metrics------------------------
mse: 45.5669
mae: 6.5164
r2: -14.3383

[0/100][150/2151][Time 10.26]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Gpt
Inf: 0.0818	Gen: 0.1462	Rec: 0.0995	E: 0.0371	R: 0.0025	P: 0.0291
--------------------Test Metrics------------------------
mse: 39.8

In [ ]:
trainer_hub.test(testset)